In [13]:
# !pip install uncurl
# !pip install certifi

In [21]:
import pandas as pd
import bs4 as BeautifulSoup
import requests, json, re

# Curl Bash

In [2]:
curlbash = """
curl 'https://www.chamberofcommerce.com/search/mapsearch?page=1' \
  -H 'Accept: application/json, text/javascript, */*; q=0.01' \
  -H 'Accept-Language: en-US,en;q=0.9' \
  -H 'Connection: keep-alive' \
  -H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' \
  -H 'Cookie: ; __adblocker=true' \
  -H 'DNT: 1' \
  -H 'Origin: https://www.chamberofcommerce.com' \
  -H 'Referer: https://www.chamberofcommerce.com/search?what=Home+Depot+Distribution+Center&where=' \
  -H 'Sec-Fetch-Dest: empty' \
  -H 'Sec-Fetch-Mode: cors' \
  -H 'Sec-Fetch-Site: same-origin' \
  -H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36' \
  -H 'X-Requested-With: XMLHttpRequest' \
  -H 'sec-ch-ua: "Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "Windows"' \
  --data-raw 'Query=Home+Depot+Distribution+Center&Bounds%5B_sw%5D%5Blng%5D=-152.18130499310706&Bounds%5B_sw%5D%5Blat%5D=4.37613941911836&Bounds%5B_ne%5D%5Blng%5D=-44.9776950068931&Bounds%5B_ne%5D%5Blat%5D=70.60745650348395&Where=' \
  --compressed
  """

# Curl Bash to JSON Function

In [14]:
    def ExtractAPI(curlbash, page):
        import uncurl, re
        results = uncurl.parse_context(curlbash)
        response = requests.post(results.url, params = {'page': f'{page}'}, cookies=results.cookies, headers=results.headers, data = results.data, verify = False)
        json_data = json.loads(response.content)
        return json_data

# JSON Extraction Function

In [15]:
def ExtractChamberOfCommerceDetails(curlbash, page):
    WEB_CC_STORE = []
    WEB_CC_LAT = []
    WEB_CC_LONG = []
    WEB_CC_ADDRESS1 = []
    WEB_CC_CITY = []
    WEB_CC_STATE = []
    WEB_CC_ZIP = []
    WEB_CC_PHONE = []

    for result in ExtractAPI(curlbash, page)['map']:
        WEB_CC_STORE.append(str(result[0]))
        WEB_CC_LAT.append(str(result[1]))
        WEB_CC_LONG.append(str(result[2]))

        # Parse address and phone number information using BeautifulSoup and regular expressions
        soup = BeautifulSoup(result[3], 'html.parser')
        address_parts = soup.find('address').get_text('\n').strip().split('\n')
        
        if len(address_parts) >= 4:
            address1 = address_parts[0].strip()
            city_state_zip = address_parts[2].strip().split(',')
            
            WEB_CC_ADDRESS1.append(address1)
            WEB_CC_CITY.append(city_state_zip[0].strip())
            
            state_zip_parts = city_state_zip[1].strip().split()
            if len(state_zip_parts) >= 2:
                WEB_CC_STATE.append(state_zip_parts[0])
                WEB_CC_ZIP.append(state_zip_parts[1])
            else:
                WEB_CC_STATE.append('')
                WEB_CC_ZIP.append('')
            
            # Extract phone number using regular expression
            phone_match = re.search(r'\b(\d{3}-\d{3}-\d{4})\b', result[3])
            if phone_match:
                WEB_CC_PHONE.append(phone_match.group(1))
            else:
                WEB_CC_PHONE.append('')
        else:
            WEB_CC_ADDRESS1.append('')
            WEB_CC_CITY.append('')
            WEB_CC_STATE.append('')
            WEB_CC_ZIP.append('')
            WEB_CC_PHONE.append('')

    data = {
        'WEB_CC_STORE': WEB_CC_STORE,
        'WEB_CC_LAT': WEB_CC_LAT,
        'WEB_CC_LONG': WEB_CC_LONG,
        'WEB_CC_ADDRESS1': WEB_CC_ADDRESS1,
        'WEB_CC_CITY': WEB_CC_CITY,
        'WEB_CC_STATE': WEB_CC_STATE,
        'WEB_CC_ZIP': WEB_CC_ZIP,
        'WEB_CC_PHONE': WEB_CC_PHONE
    }

    df_CC_WEB = pd.DataFrame(data)
    return df_CC_WEB

# Get Final Results

In [1]:
df_CC_WEB = pd.DataFrame(columns=['WEB_CC_STORE', 'WEB_CC_LAT', 'WEB_CC_LONG', 'WEB_CC_ADDRESS1', 'WEB_CC_CITY', 'WEB_CC_STATE', 'WEB_CC_ZIP', 'WEB_CC_PHONE'])

def append_data_to_df(curlbash, page, df):
    result = ExtractChamberOfCommerceDetails(curlbash, page=page)
    if not result.empty and 'WEB_CC_STORE' in result.columns:
        df = df.append(result, ignore_index=True)
        page += 1
    else:
        page = None
    return df, page
page = 1
while page is not None:
    df_CC_WEB, page = append_data_to_df(curlbash, page, df_CC_WEB)

print(len(df_CC_WEB))
df_CC_WEB

In [2]:
# append_data_to_df(curlbash = curlbash, page = 1, df = df_CC_WEB)
ExtractChamberOfCommerceDetails(curlbash, page=1)